In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tkinter import Tk, filedialog, messagebox
from datetime import datetime

In [2]:
try:
    import tifffile
except ImportError:
    messagebox.showerror("Error", "tifffile package not found. Install via 'pip install tifffile imagecodecs'.")
    exit()

In [3]:
Tk().withdraw()

''

In [4]:
folder1 = filedialog.askdirectory(title="Select First Folder")
folder2 = filedialog.askdirectory(title="Select Second Folder")
if not folder1 or not folder2:
    messagebox.showerror("Error", "Both folders must be selected.")
    exit()

In [5]:
csv_path = r"C:\Users\Admin\OneDrive - Indian Institute of Science (1)\Testing Folder\V3_transform.csv"
if not os.path.exists(csv_path):
    messagebox.showerror("Error", f"CSV file not found:\n{csv_path}")
    exit()

In [6]:
df = pd.read_csv(csv_path)
required_cols = {
    "rotation", "scale", "offset_x", "offset_y",
    "warp_tl_x", "warp_tl_y", "warp_tr_x", "warp_tr_y",
    "warp_bl_x", "warp_bl_y", "warp_br_x", "warp_br_y",
    "diag1", "diag2", "hstretch", "vstretch",
    "top_tilt_x", "top_tilt_y", "bottom_tilt_x", "bottom_tilt_y"
}
if not required_cols.issubset(df.columns):
    messagebox.showerror("Error", "CSV missing required columns.")
    exit()
params = df.iloc[0]

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\OneDrive - Indian Institute of Science (1)\\Testing Folder\\V3_transform.csv'

In [ ]:
common_parent = os.path.commonpath([folder1, folder2])
folder2_name = os.path.basename(folder2)

In [ ]:
output_root = os.path.join(common_parent, f"{folder2_name}_transformed")
os.makedirs(output_root, exist_ok=True)

In [ ]:
exclude_files = ['Qa_vs_centroid.tif', 'Qa_vs_peak.tif']

In [ ]:
def get_tif_files(folder):
    return [f for f in os.listdir(folder)
            if f.lower().endswith(('.tif', '.tiff')) and f not in exclude_files]

files1 = get_tif_files(folder1)
files2 = get_tif_files(folder2)
matches = []
for f1 in files1:
    name1 = os.path.splitext(f1)[0]
    if len(name1) < 3:
        continue
    last3_a = name1[-3:]
    for f2 in files2:
        name2 = os.path.splitext(f2)[0]
        if len(name2) < 3:
            continue
        last3_b = name2[-3:]
        if last3_a == last3_b:
            matches.append((f1, f2))

if not matches:
    messagebox.showinfo("Info", "No matching images found.")
    exit()

In [ ]:
def transform_image(image, params):
    rows, cols = image.shape[:2]
    center = (cols // 2, rows // 2)

    M_rot = cv2.getRotationMatrix2D(center, params["rotation"], params["scale"])
    image_rot = cv2.warpAffine(image, M_rot, (cols, rows))

    tl = [params["warp_tl_x"], params["warp_tl_y"]]
    tr = [params["warp_tr_x"], params["warp_tr_y"]]
    bl = [params["warp_bl_x"], params["warp_bl_y"]]
    br = [params["warp_br_x"], params["warp_br_y"]]

    tl[0] -= params["diag1"]; tl[1] -= params["diag1"]
    br[0] += params["diag1"]; br[1] += params["diag1"]
    tr[0] += params["diag2"]; tr[1] -= params["diag2"]
    bl[0] -= params["diag2"]; bl[1] += params["diag2"]

    h, v = params["hstretch"], params["vstretch"]
    tl[0] -= h; bl[0] -= h
    tr[0] += h; br[0] += h
    tl[1] -= v; tr[1] -= v
    bl[1] += v; br[1] += v

    tl[0] += params["top_tilt_x"]; tr[0] += params["top_tilt_x"]
    tl[1] += params["top_tilt_y"]; tr[1] += params["top_tilt_y"]
    bl[0] += params["bottom_tilt_x"]; br[0] += params["bottom_tilt_x"]
    bl[1] += params["bottom_tilt_y"]; br[1] += params["bottom_tilt_y"]

    pts1 = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
    pts2 = np.float32([
        [0 + tl[0], 0 + tl[1]],
        [cols + tr[0], 0 + tr[1]],
        [0 + bl[0], rows + bl[1]],
        [cols + br[0], rows + br[1]]
    ])
    M_persp = cv2.getPerspectiveTransform(pts1, pts2)
    image_warped = cv2.warpPerspective(image_rot, M_persp, (cols, rows))

    M_offset = np.float32([[1, 0, params["offset_x"]], [0, 1, params["offset_y"]]])
    image_final = cv2.warpAffine(image_warped, M_offset, (cols, rows))

    return image_final

for f1, f2 in matches:
    path2 = os.path.join(folder2, f2)

    try:
        img2 = tifffile.imread(path2)
    except Exception as e:
        print(f"Failed to read {f2}: {e}")
        continue

    if len(img2.shape) == 2:
        img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    transformed = transform_image(img2, params)

    name2 = os.path.splitext(f2)[0]
    save_name = f"{name2}_transformed.tif"
    save_path = os.path.join(output_root, save_name)

    tifffile.imwrite(save_path, transformed)
    print(f"Saved transformed image: {save_path}")

messagebox.showinfo("Done", f"All matched images transformed and saved!\n\nSaved in:\n{output_root}")